In [3]:
import pandas as pd
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv("partidos_colombia_2025.csv")

In [4]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Fecha                   1910 non-null   object
 1   Equipo Local            1910 non-null   object
 2   Marcador                1910 non-null   object
 3   Equipo Visitante        1910 non-null   object
 4   Tarjeta Roja Local      1910 non-null   int64 
 5   Tarjeta Roja Visitante  1910 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 89.7+ KB
None
        Fecha          Equipo Local Marcador        Equipo Visitante  \
0  26-01-2025     Alianza Petrolera    1 - 1  Independiente Medellín   
1  02-02-2025     Deportivo Pereira    0 - 0       Alianza Petrolera   
2  07-02-2025       Deportes Tolima    3 - 1       Alianza Petrolera   
3  17-02-2025  Atlético Bucaramanga    0 - 0       Alianza Petrolera   
4  22-02-2025     Alianza Petrolera    3 - 2       Atlético Na

## Haga un resumen estadístico de cada equipo, donde se vea tabla de frecuencias e indicadores estadísticos.

In [23]:
import pandas as pd

# ====== Extraer los goles del marcador ======
df[["Goles Local", "Goles Visitante"]] = df["Marcador"].str.split(" - ", expand=True).astype(int)

# ====== Calcular los resultados para cada equipo ======
def obtener_resultado(goles_a_favor, goles_en_contra):
    if goles_a_favor > goles_en_contra:
        return "Victoria"
    elif goles_a_favor < goles_en_contra:
        return "Derrota"
    else:
        return "Empate"

df["Resultado Local"] = df.apply(lambda row: obtener_resultado(row["Goles Local"], row["Goles Visitante"]), axis=1)
df["Resultado Visitante"] = df.apply(lambda row: obtener_resultado(row["Goles Visitante"], row["Goles Local"]), axis=1)

# ====== Seleccionar y renombrar las columnas ======
df_total = df[[
     "Fecha",
    "Equipo Local", "Equipo Visitante", "Marcador", 
    "Goles Local", "Goles Visitante", 
    "Tarjeta Roja Local", "Tarjeta Roja Visitante",
    "Resultado Local", "Resultado Visitante"
]].copy()

# Renombrar columnas para mayor claridad
df_total.rename(columns={
    "Equipo Local": "Local", 
    "Equipo Visitante": "Visitante",
    "Tarjeta Roja Local": "Tarjetas Rojas Local",
    "Tarjeta Roja Visitante": "Tarjetas Rojas Visitante"
}, inplace=True)


In [24]:
df_total

,Fecha,Local,Visitante,Marcador,Goles Local,Goles Visitante,Tarjetas Rojas Local,Tarjetas Rojas Visitante,Resultado Local,Resultado Visitante
0,26-01-2025,Alianza Petrolera,Independiente Medellín,1 - 1,1,1,0,0,Empate,Empate
1,02-02-2025,Deportivo Pereira,Alianza Petrolera,0 - 0,0,0,0,0,Empate,Empate
2,07-02-2025,Deportes Tolima,Alianza Petrolera,3 - 1,3,1,0,0,Victoria,Derrota
3,17-02-2025,Atlético Bucaramanga,Alianza Petrolera,0 - 0,0,0,0,0,Empate,Empate
4,22-02-2025,Alianza Petrolera,Atlético Nacional,3 - 2,3,2,0,0,Victoria,Derrota
...,...,...,...,...,...,...,...,...,...,...
1905,12-10-2023,Deportivo Pasto,Union Magdalena,3 - 1,3,1,0,0,Victoria,Derrota
1906,18-10-2023,Millonarios,Union Magdalena,1 - 1,1,1,0,0,Empate,Empate
1907,20-10-2023,Union Magdalena,Bucaramanga,1 - 2,1,2,0,0,Derrota,Victoria
1908,27-10-2023,Deportes Tolima,Union Magdalena,2 - 1,2,1,0,0,Victoria,Derrota


In [25]:
# Guardar en un archivo CSV
df_total.to_csv("partidos_colombia.csv", index=False, encoding="utf-8")

print("✅ Archivo CSV guardado exitosamente.")

✅ Archivo CSV guardado exitosamente.


In [16]:
# Contar resultados según condición (Local/Visitante)
print(df_total.groupby("Condición")["Resultado"].value_counts())


# Contar cuántas veces aparece cada partido (ida y vuelta)
duplicados = df_total.groupby(["Fecha", "Equipo", "Rival"]).size().reset_index(name="conteo")

# Si hay partidos que no aparecen dos veces, hay un problema
print(duplicados["conteo"].value_counts())

# Verificar si el resultado visitante se asignó correctamente
df_visitante_check = df_total[df_total["Condición"] == "Visitante"]
df_visitante_check[["Equipo", "Rival", "Resultado Local", "Resultado"]].head(10)



Condición  Resultado
Local      Victoria     902
           Empate       568
           Derrota      440
Visitante  Derrota      902
           Empate       568
           Victoria     440
Name: count, dtype: int64
conteo
2    1910
Name: count, dtype: int64


,Equipo,Rival,Resultado Local,Resultado
1910,Independiente Medellín,Alianza Petrolera,Empate,Empate
1911,Alianza Petrolera,Deportivo Pereira,Empate,Empate
1912,Alianza Petrolera,Deportes Tolima,Victoria,Derrota
1913,Alianza Petrolera,Atlético Bucaramanga,Empate,Empate
1914,Atlético Nacional,Alianza Petrolera,Victoria,Derrota
1915,Alianza Petrolera,Once Caldas,Derrota,Victoria
1916,Unión Magdalena,Alianza Petrolera,Victoria,Derrota
1917,Llaneros,América de Cali,Victoria,Derrota
1918,América de Cali,Atlético Bucaramanga,Derrota,Victoria
1919,Deportivo Pasto,América de Cali,Empate,Empate


In [ ]:
# Tabla de frecuencia para el resultado del equipo
frecuencia_resultado = df_total['Resultado'].value_counts().reset_index()
frecuencia_resultado.columns = ['Resultado', 'Frecuencia']

# Tabla de frecuencia para la condición (Local/Visitante)
frecuencia_condicion = df_total['Condición'].value_counts().reset_index()
frecuencia_condicion.columns = ['Condición', 'Frecuencia']

# Tabla de frecuencia para tarjetas rojas
frecuencia_tarjetas = df_total['Tarjetas Rojas'].value_counts().reset_index()
frecuencia_tarjetas.columns = ['Tarjetas Rojas', 'Frecuencia']

# Tabla de frecuencia para los goles anotados
frecuencia_goles = df_total['Goles a Favor'].value_counts().reset_index()
frecuencia_goles.columns = ['Goles a Favor', 'Frecuencia'] 

# Mostrar las tablas
print("Frecuencia de Resultados:")
print(frecuencia_resultado)

print("\nFrecuencia de Condición (Local/Visitante):")
print(frecuencia_condicion)

print("\nFrecuencia de Tarjetas Rojas:")
print(frecuencia_tarjetas)

print("\nFrecuencia de Goles Anotados:")
print(frecuencia_goles)

print("\nFrecuencia de Goles Anotados:")
print(frecuencia_goles)

Indicadores Estadísticos para 'Resultado':
Media: 1.35
Mediana: 1.0
Moda: 0
Desviación Estándar: 1.28
Varianza: 1.63
Mínimo: 0
Máximo: 3


In [18]:
import pandas as pd

# Asignar valores numéricos al resultado
mapeo_resultado = {'Victoria': 3, 'Empate': 1, 'Derrota': 0}
df_total['Resultado_Numerico'] = df_total['Resultado'].map(mapeo_resultado)

# 📊 Tabla de Frecuencia
frecuencia_resultado = df_total['Resultado'].value_counts().reset_index()
frecuencia_resultado.columns = ['Resultado', 'Frecuencia']

# 📈 Indicadores Estadísticos
media = df_total['Resultado_Numerico'].mean()
mediana = df_total['Resultado_Numerico'].median()
moda = df_total['Resultado_Numerico'].mode()[0]  # Puede haber más de una moda
desviacion = df_total['Resultado_Numerico'].std()
varianza = df_total['Resultado_Numerico'].var()
minimo = df_total['Resultado_Numerico'].min()
maximo = df_total['Resultado_Numerico'].max()

# 📋 Mostrar resultados
print("📊 Tabla de Frecuencia:")
print(frecuencia_resultado)
print("\n📈 Indicadores Estadísticos para 'Resultado':")
print(f"Media: {media:.2f}")
print(f"Mediana: {mediana}")
print(f"Moda: {moda}")
print(f"Desviación Estándar: {desviacion:.2f}")
print(f"Varianza: {varianza:.2f}")
print(f"Mínimo: {minimo}")
print(f"Máximo: {maximo}")


📊 Tabla de Frecuencia:
  Resultado  Frecuencia
0  Victoria        1342
1   Derrota        1342
2    Empate        1136

📈 Indicadores Estadísticos para 'Resultado':
Media: 1.35
Mediana: 1.0
Moda: 0
Desviación Estándar: 1.28
Varianza: 1.63
Mínimo: 0
Máximo: 3


In [20]:
# Filtrar solo los datos del año 2025
df_2025 = df_total[df_total['Fecha'].dt.year == 2025].copy()

# Mapeo de resultados a puntos
mapeo_resultado = {'Victoria': 3, 'Empate': 1, 'Derrota': 0}
df_2025['Puntos'] = df_2025['Resultado'].map(mapeo_resultado)

# 🏆 **1. Probabilidad de Once Caldas de ser campeón**
puntos_totales = df_2025.groupby('Equipo')['Puntos'].sum()
puntos_once_caldas = puntos_totales.get('Once Caldas', 0)
probabilidad_once_caldas = puntos_once_caldas / puntos_totales.sum() * 100

# 🏆 **2. Equipo con más puntos (campeón del año)**
campeon_anual = puntos_totales.idxmax()
puntos_campeon = puntos_totales.max()

# 🏟️ **3. Mejor equipo de local**
locales = df_2025[df_2025['Condición'] == 'Local'].groupby('Equipo')['Puntos'].sum()
mejor_local = locales.idxmax()
puntos_mejor_local = locales.max()

# 🚍 **4. Mejor equipo de visitante**
visitantes = df_2025[df_2025['Condición'] == 'Visitante'].groupby('Equipo')['Puntos'].sum()
mejor_visitante = visitantes.idxmax()
puntos_mejor_visitante = visitantes.max()

# ⚽ **5. Equipo que más goles anotó y que más recibió**
goles_favor = df_2025.groupby('Equipo')['Goles a Favor'].sum()
goles_contra = df_2025.groupby('Equipo')['Goles en Contra'].sum()

equipo_mas_goleador = goles_favor.idxmax()
goles_maximos = goles_favor.max()

equipo_mas_goleado = goles_contra.idxmax()
goles_recibidos_maximos = goles_contra.max()

# 📋 **Mostrar respuestas**
print("📌 Probabilidad de Once Caldas de ser campeón:", f"{probabilidad_once_caldas:.2f}%")
print("🏆 Campeón del año:", campeon_anual, f"con {puntos_campeon} puntos")
print("🏟️ Mejor equipo de local:", mejor_local, f"con {puntos_mejor_local} puntos")
print("🚍 Mejor equipo de visitante:", mejor_visitante, f"con {puntos_mejor_visitante} puntos")
print("⚽ Equipo más goleador:", equipo_mas_goleador, f"con {goles_maximos} goles")
print("🛑 Equipo más goleado:", equipo_mas_goleado, f"con {goles_recibidos_maximos} goles")


📌 Probabilidad de Once Caldas de ser campeón: 5.88%
🏆 Campeón del año: Independiente Medellín con 36 puntos
🏟️ Mejor equipo de local: América de Cali con 20 puntos
🚍 Mejor equipo de visitante: Independiente Medellín con 16 puntos
⚽ Equipo más goleador: Atlético Nacional con 36 goles
🛑 Equipo más goleado: Envigado con 30 goles
